In [ ]:
import pandas as pd
import numpy as np
import sys, os
sys.path.append(os.getcwd() + '/..')
from utils import mkdir, submission_summary, needs_size_filter, size_filter, plot_map, plot_date_hist, maps, open_tar_shapefile, open_zip_shapefile

In [ ]:
# go down from rgi7_scripts/workflow
data_dir = '../../../rgi7_data/'

l4_dir = os.path.join(data_dir, 'l4_rgi7b0')

In [ ]:
reg_file = open_zip_shapefile(os.path.join(data_dir, '00_rgi70_regions/00_rgi70_O1Regions.zip'))
sreg_file = open_zip_shapefile(os.path.join(data_dir, '00_rgi70_regions/00_rgi70_O2Regions.zip'))

In [ ]:
contrib = {}
for reg in range(1, 20):
    regf = reg_file.loc[reg_file.o1region == f'{reg:02d}'].iloc[0]
    rd = 'RGI2000-v7.0-G-' + regf.long_code
    shp = pd.read_csv(os.path.join(l4_dir, rd, rd + '_subm_info.csv'))
    for i, s in shp.iterrows():
        sub = s.submitters.strip()
        if sub not in contrib:
            contrib[sub] = {}
            contrib[sub]['subm_id'] = []
            contrib[sub]['rgi_reg'] = []
            contrib[sub]['area_km2'] = 0
            contrib[sub]['n_outlines'] = 0

        contrib[sub]['subm_id'].append(str(s.subm_id))
        contrib[sub]['rgi_reg'].append(regf.o1region)
        contrib[sub]['area_km2'] += s.area_km2
        contrib[sub]['n_outlines'] += s.n_outlines

        for analyst in s.analysts.split(';'):
            analyst = analyst.strip()
            if analyst == sub:
                continue
            if analyst not in contrib:
                contrib[analyst] = {}
                contrib[analyst]['subm_id'] = []
                contrib[analyst]['rgi_reg'] = []
                contrib[analyst]['area_km2'] = 0
                contrib[analyst]['n_outlines'] = 0

            contrib[analyst]['subm_id'].append(str(s.subm_id))
            contrib[analyst]['rgi_reg'].append(regf.o1region)
            contrib[analyst]['area_km2'] += s.area_km2
            contrib[analyst]['n_outlines'] += s.n_outlines

In [ ]:
for key, value in sorted(contrib.items()):
    print(f'- **{key}**', end=' (')
    print(f'Region: ' + ', '.join(np.unique(value['rgi_reg'])), end='. ')
    print(f'Subm ID: ' + ', '.join(np.unique(value['subm_id'])), end='. ')
    print(f"N outlines: {value['n_outlines']}. Area: {round(value['area_km2'], 1):.1f} km²)")